# Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scikitplot as skplt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Carregamento de datasets

In [ ]:
# Carregando datasets
df_treino = pd.read_csv('../input/costa-rican-household-poverty-prediction/train.csv')
df_teste = pd.read_csv('../input/costa-rican-household-poverty-prediction/test.csv')

df_treino.shape, df_teste.shape

Dataset de treino com 9.557 observações e 143 variáveis.
<br>Dataset de teste com 23.856 observações e 142 variáveis.

In [ ]:
# Visualizando dados de df
df_treino.info(max_cols=145)

In [ ]:
# Visualizando dados de test
df_teste.info(max_cols=145)

# Missing values

In [ ]:
# Verificando os missing values
df_treino.isnull().sum()[df_treino.isnull().sum() >= 1] 

In [ ]:
# Excluindo os missing values
missing = list(df_treino.isnull().sum()[df_treino.isnull().sum() >= 1].index) 
df_treino.drop([missing.pop(0) for i in range(3)], axis =1, inplace = True)
df_treino.shape

In [ ]:
# Preenchendo os missing values com a média
df_treino[missing] = df_treino[missing].fillna(np.mean)

In [ ]:
# Verificando a inexistência de missing values após tratamento
df_treino.isnull().sum()[df_treino.isna().sum() >= 1]

In [ ]:
# Excluindo variáveis categóricas da base de treino
variaveis = list(df_treino.select_dtypes('int64').columns)

binarias = []
for i in variaveis:
  u =df_treino[i].unique()

  if u.sum() == 1:
    binarias.append(i)

discretas = [i for i in variaveis if i not in binarias]

df_treino = df_treino[binarias + discretas]

# Análise Exploratória

In [ ]:
# Sexo dos ocupantes das residências
var = 'male','female'
leg = ['Masculino','Feminino']

y = []
for v in var: 
    y.append(df_treino[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Sexo")
plt.show()

Observamos uma leve predominância de mulheres entre os ocupantes.

In [ ]:
# Idade dos ocupantes das residências
plt.figure(figsize=(18,5))
sns.histplot(data=df_treino, x="age").set_title("Idade")
plt.show()

Observamos uma maioria de jovens entre os ocupantes.

In [ ]:
# Parentesco dos ocupantes das residências
var = 'parentesco1','parentesco2','parentesco3','parentesco4','parentesco5','parentesco6','parentesco7','parentesco8','parentesco9','parentesco10','parentesco11','parentesco12'
leg = ['Chefe de família','Cônjuge','Filho(a)','Enteado(a)','Genro/Nora','Neto(a)','Pai/Mãe','Sogro(a)','Irmão/Irmã','Cunhado(a)','Outro familiar','Outro não familiar']

y = []
for v in var: 
    y.append(df_treino[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Parentesco")
plt.show()

As casas são ocupadas majoritariamente pelo(a) chefe de família, cônjuge e filhos. Também há um bom número de netos entre os ocupantes.

In [ ]:
# Estado civil dos ocupantes das residências
var = 'estadocivil1','estadocivil2','estadocivil3','estadocivil4','estadocivil5','estadocivil6','estadocivil7'
leg = ['Menores de 10 anos','União estável','Casados(as)','Divorciados(as)','Separados(as)','Viúvos(as)','Solteiros(as)']

y = []
for v in var: 
    y.append(df_treino[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Estado Civil")
plt.show()

Há uma maioria de solteiros e casados entre os ocupantes.

In [ ]:
# Grau de instrução dos ocupantes das residências
var = 'instlevel1','instlevel2','instlevel3','instlevel4','instlevel5','instlevel6','instlevel7', 'instlevel8', 'instlevel9'
leg = ['Nenhum','Primário incompleto','Primário completo','Secundário incompleto','Secundário completo','Técnico incompleto','Técnico completo','Graduação','Pós-Graduação ou superior']

y = []
for v in var: 
    y.append(df_treino[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Grau de Instrução")
plt.show()

A maioria dos ocupantes têm baixo grau de instrução (nenhum, primário e secundário incompleto). A exceção fica para os graduandos ou graduados.

In [ ]:
# Tipo de moradia
var = 'tipovivi1','tipovivi2','tipovivi3','tipovivi4','tipovivi5'
leg = ['Própria','Financiada','Alugada','Precária','Outras']

y = []
for v in var: 
    y.append(df_treino[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Tipo de Moradia")
plt.show()

A maioria das moradias são próprias, ou seja, o morador tem sua propriedade integral, não alienada.

In [ ]:
# Número de cômodos das residências
plt.figure(figsize=(18,5))
sns.histplot(data=df_treino, x="rooms").set_title("Número de Cômodos")
plt.show()

A maior parte das residências possuem entre 4 e 6 cômodos.

In [ ]:
# Imóvel rural ou urbano
var = 'area2','area1'
leg = ['Rural','Urbano']

y = []
for v in var: 
    y.append(df_treino[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Imóvel rural x Imóvel urbano")
plt.show()

A maior parte das residências é urbana.

In [ ]:
# Superlotação (por quarto e por cômodo)
var = 'hacdor','hacapo'
leg = ['Por quarto','Por cômodo']

y = []
for v in var: 
    y.append(df_treino[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Superlotação - Por quarto ou por cômodo")
plt.show()

O gráfico acima mostra como as residências foram definidas como superlotadas, quer tenha sido a avaliação por quarto ou por cômodo.

# Balanceamento de classes (Random Over Sampler) e Random Forest

In [ ]:
# Verificando a distribuição das classes na variável Target
df_treino["Target"].value_counts(normalize = True)

In [ ]:
# Balanceando as classes da variável target a partir de amostra com Random Over Sampler
sample = RandomOverSampler(random_state = 42)
X_over, y_over = sample.fit_resample(df_treino, df_treino["Target"])
df_novo = pd.DataFrame(X_over)
df_novo["Target"] = y_over

In [ ]:
# Verificando a distribuição das classes na variável Target após balanceamento
df_novo["Target"].value_counts(normalize = True)

In [ ]:
# Separando variáveis para treinamento
feats = [c for c in df_novo.columns if c not in ['Id', 'Target']]

In [ ]:
train, test = df_novo, df_teste
rf = RandomForestClassifier(n_jobs=1, n_estimators=200, random_state=42)
rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [ ]:
## Matriz de confusão
pd.crosstab(train['Target'],test['Target'])

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Verificando o peso de cada coluna na predição
fig = plt.figure(figsize=(15, 20))
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

<b>Resultado ROS:</b> 0.36488, superior à obtida antes do balanceamento (0.36353). 

# Balanceamento de classes (Random Under Sampler) e Random Forest

In [ ]:
# Balanceando as classes da variável target a partir de amostra com Random Under Sampler
sample = RandomUnderSampler(random_state = 42)
X_under, y_under = sample.fit_resample(df_treino, df_treino["Target"])
df_novo2 = pd.DataFrame(X_under)
df_novo2["Target"] = y_under

In [ ]:
# Verificando a distribuição das classes na variável Target após balanceamento
df_novo2["Target"].value_counts(normalize = True)

In [ ]:
train, test = df_novo2, df_teste
rf2 = RandomForestClassifier(n_jobs=1, n_estimators=200, random_state=42)
rf2.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf2.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
## Matriz de confusão
pd.crosstab(train['Target'],test['Target'])

<b>Resultado RUS:</b> 0.40562, superior à obtida antes do balanceamento (0.36353). 
<br><br> Determinado o Random Under Sampling como o melhor método de balanceamento.

# Testes com mudanças de parâmetros

In [ ]:
# Criando um novo modelo
rf3 = RandomForestClassifier(n_jobs=2, n_estimators=200, random_state=42)
rf3.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf3.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)

<b>Resultado:</b> 0.40562
<br><br> Sem alterações

In [ ]:
# Criando um novo modelo
rf4 = RandomForestClassifier(n_jobs=1, n_estimators=400, random_state=42)
rf4.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf4.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)

<b>Resultado:</b> 0.40562
<br><br> Sem alterações

# Gradient Boosting

In [ ]:
# Testando com Gradient Boosting

gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)
gbm.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = gbm.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)

<b>Resultado:</b> 0.36961, superior à obtida antes do balanceamento (0.36353), mas inferior ao resultado de Random Forest. 

# XGBoost

In [ ]:
# Testando com XGBoost
xgb = XGBClassifier(n_estimators=200, learning_rate=0.09, random_state=42)
xgb.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = xgb.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)

<b>Resultado:</b> 0.36509, superior à obtida antes do balanceamento (0.36353), mas inferior ao resultado de Random Forest. 

# AdaBoost

In [ ]:
# Testando com AdaBoost
abc = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, random_state=42)
abc.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = abc.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)

<b>Resultado:</b> 0.36509, superior à obtida antes do balanceamento (0.36353), mas inferior ao resultado de Random Forest. 

# CatBoost

In [ ]:
# Testando com CatBoost
cbc = CatBoostClassifier(random_state=42)
cbc.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = cbc.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [ ]:
test[['Id', 'Target']].to_csv('submission.csv', index=False)

<b>Resultado:</b> 0.39720, superior à obtida antes do balanceamento (0.36353), mas inferior ao resultado de Random Forest. 

# Resultado final
Random Forest (RUS): 0.40562
<br> CatBoost (RUS): 0.39720
<br> Gradient Boosting (RUS): 0.36961
<br> AdaBoost (RUS): 0.36509
<br> XGBoost (RUS): 0.36509
<br> Random Forest (ROS): 0.36488

Resultado referência (Random Forest sem balanceamento): 0.36353